In [ ]:
% matplotlib notebook
import bokeh.plotting as bp
import bokeh.models as bm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gatos as dmc

In [ ]:
directorio_resultados = '../tmp/'
archivo = 'distribucion_posterior_repeticion_2_iteraciones_1000000.csv'
datos = pd.read_csv(directorio_resultados + archivo)

## CDF éxito de erradicación  

In [ ]:
probabilidad = []
remanentes_capturados = []
muestra = 100_000
for iEsfuerzo in range(1,1001):
    No = datos.No.sample(n=muestra, replace=True).values.astype(np.int32) - 606
    beta = datos.b.sample(n=muestra, replace=True).values
    alpha = datos.a.sample(n=muestra, replace=True).values
    esfuerzo = iEsfuerzo * 0.1
    exp = np.exp(alpha + beta * esfuerzo)
    p = exp/(1+exp)
    probabilidad.append(1-sum(np.random.binomial(No,p) < No)/muestra) 
    remanentes_capturados.append(np.mean(np.random.binomial(No,p)))

In [ ]:
esfuerzo_trampas = np.array(range(1,1001,1))*0.1 * (5 * 30 * 7)
fuente = bp.ColumnDataSource(
    data=dict(
        esfuerzo_trampas=esfuerzo_trampas,
        remanentes_capturados=remanentes_capturados
    )
)
encima = bm.HoverTool(
    tooltips=[
        ("Capturas", "@remanentes_capturados"),
        ("Esfuerzo", "@esfuerzo_trampas")], 
    mode='vline'
)
herramientas = "pan,box_zoom,wheel_zoom,reset,save"
fig = bp.figure(title="Capturas promedio de individuos remanentes en función del esfuerzo",
                x_axis_label="Esfuerzo (noches*trampas)", y_axis_label='Capturas (individuos)',
                x_range=(0,50000), tools=[encima, herramientas])
fig.line(x='esfuerzo_trampas', y='remanentes_capturados', source=fuente)
#bp.show(fig)

In [ ]:
paso = 20
remanentes_derivar = np.convolve(remanentes_capturados, np.ones((paso,))/paso, mode='valid')
esfuerzo_graficar = np.convolve(esfuerzo_trampas, np.ones((paso,))/paso, mode='valid')

In [ ]:
df = np.diff(remanentes_derivar)
dx = esfuerzo_graficar[paso] - esfuerzo_graficar[0]
Df = df/dx

In [ ]:
fuente = bp.ColumnDataSource(
    data=dict(
        derivada=Df,
        esfuerzo_graficar=(esfuerzo_graficar[1:]+esfuerzo_graficar[:-1])/2
    )
)
encima = bm.HoverTool(
    tooltips=[
        ("Capturas", "@derivada"),
        ("Esfuerzo", "@esfuerzo_graficar")], 
    mode='vline'
)
fig_derivada = bp.figure(title="Derivada capturas promedio de individuos remanentes en función del esfuerzo",
                x_axis_label="Esfuerzo (noches*trampas)", y_axis_label='Capturas (individuos)',
                x_range=(0,50000), tools=[encima, herramientas])

fig_derivada.line(x='esfuerzo_graficar',y='derivada', source=fuente)

In [ ]:
p = bp.gridplot([[fig_derivada, fig]])
bp.show(p)

In [ ]:
p1 = bp.figure(title="cdf de éxito de erradicación", x_axis_label="Esfuerzo (noches*trampas)", y_axis_label='Probabilidad',\
            y_range=(0, 1),x_range=(0, 100000))
p1.grid.visible = False
p1.line(esfuerzo_trampas,probabilidad)
p1.line(esfuerzo_trampas,marca_95, line_color="#D95B43")
bp.output_file('../resultados/cdf_exito_erradicacion.html', title="cdf éxito erradicacion")
bp.show(p1)

## PDF éxito de erradicación 
Calculamos la función de distribución de probabilidad derivando de la {\it CDF}.  

In [ ]:
dt = 20
derivada = np.array(probabilidad[0:-1:dt])
derivada[1:] -= derivada[:-1].copy()
esfuerzo_trampas_menor_paso = esfuerzo_trampas[0:-1:dt]  # Cambio de paso

In [ ]:
plt.bar(esfuerzo_trampas_menor_paso,derivada,width=2000)

In [ ]:
p2 = bp.figure(title="pdf de éxito de erradicación", x_axis_label='Esfuerzo (trampas * noche)', y_axis_label='Probabilidad', \
            y_range=(0,0.12))
p2.quad(top=derivada, bottom=0, left=esfuerzo_trampas_menor_paso, right=esfuerzo_trampas_menor_paso+2000)
p2.grid.visible = False

In [ ]:
bp.output_file('../resultados/pdf_exito_erradicacion.html', title="pdf éxito de erradicación")
bp.show(p2)

In [ ]:
id_min = len(np.array(probabilidad)) - sum(np.array(probabilidad)>0.025)
id_max = len(np.array(probabilidad)) - sum(np.array(probabilidad)>0.975)
((esfuerzo_trampas[id_min]+ esfuerzo_trampas[id_min+1])/2,(esfuerzo_trampas[id_max]+esfuerzo_trampas[id_max+1])/2)